In [11]:
import pandas as pd
import os
import math

# Define models
gpu_a100_models = ["ccmr", "flowformer", "flowformer++", "dip", "ms_raft+"]

# Load CSV
df = pd.read_csv('missing_combinations.csv', delimiter=',')

# Paths to script blueprints
script_blueprint_path = "/hkfs/work/workspace/scratch/ma_jcaspary-ma_jcaspary-team_project_fss2024/benchmarking_robustness/optical_flow_estimation/scripts/evaluation_minimum/automated_script/automated_script/automatic_script_blueprint.sh"
script_blueprint_path_h100 = "/hkfs/work/workspace/scratch/ma_jcaspary-ma_jcaspary-team_project_fss2024/benchmarking_robustness/optical_flow_estimation/scripts/evaluation_minimum/automated_script/automated_script/automatic_script_blueprint_h100.sh"

# Subset where df["model"] is in gpu_a100_models
df_in_a100_models = df[df["model"].isin(gpu_a100_models)]

# Subset where df["model"] is not in gpu_a100_models
df_not_in_a100_models = df[~df["model"].isin(gpu_a100_models)]

# Read the original blueprint script content
with open(script_blueprint_path, 'r') as original_script:
    original_content = original_script.readlines()

# Split the header (lines starting with '#SBATCH') and the rest of the script
header_lines = []
rest_of_script = []
header_complete = False

for line in original_content:
    if line.startswith("#SBATCH") or line.startswith("#!/bin/bash") or not header_complete:
        header_lines.append(line)
        if line.strip() == "":
            header_complete = True
    else:
        rest_of_script.append(line)

# Initialize an empty list for combinations
combinations_list = []

# Iterate over the rows of df_not_in_a100_models to build the combinations
for index, row in df_not_in_a100_models.iterrows():
    attack_arguments_dict = {}
    row_dict = row.to_dict()
    
    # Build attack arguments
    attack_arguments_dict["val_dataset"] = row_dict.get("dataset", "nan")
    attack_arguments_dict["attack"] = row_dict.get("attack", "nan")
    attack_arguments_dict["attack_targeted"] = row_dict.get("targeted", "nan")
    attack_arguments_dict["attack_target"] = row_dict.get("target", "nan")
    attack_arguments_dict["attack_loss"] = row_dict.get("loss", "nan")
    attack_arguments_dict["attack_epsilon"] = row_dict.get("epsilon", "nan")

    # Adjust epsilon values based on conditions
    if attack_arguments_dict["attack_epsilon"] == 0.0314:
        attack_arguments_dict["attack_epsilon"] = round(8.0/255.0, 10)
    elif attack_arguments_dict["attack_epsilon"] == 0.0157:
        attack_arguments_dict["attack_epsilon"] = round(4.0/255.0, 10)
    elif attack_arguments_dict["attack_epsilon"] == 0.251:
        attack_arguments_dict["attack_epsilon"] = round(64.0/255.0, 10)

    attack_arguments_dict["attack_alpha"] = row_dict.get("alpha", "nan")
    attack_arguments_dict["attack_iterations"] = row_dict.get("iterations", "nan")
    attack_arguments_dict["attack_norm"] = row_dict.get("norm", "nan")
    attack_arguments_dict["cc_name"] = row_dict.get("name", "nan")
    attack_arguments_dict["cc_severity"] = row_dict.get("severity", "nan")
    attack_arguments_dict["attack_optim_target"] = row_dict.get("optim", "nan")
    attack_arguments_dict["3dcc_corruption"] = row_dict.get("3dcc_corruption", "nan")
    attack_arguments_dict["3dcc_intensity"] = row_dict.get("3dcc_intensity", "nan")

    if attack_arguments_dict["attack_iterations"] != "nan" and not math.isnan(float(attack_arguments_dict["attack_iterations"])):
        attack_arguments_dict["attack_iterations"] = int(attack_arguments_dict["attack_iterations"])
    if attack_arguments_dict["cc_severity"] != "nan" and not math.isnan(float(attack_arguments_dict["cc_severity"])):
        attack_arguments_dict["cc_severity"] = int(attack_arguments_dict["cc_severity"])
    if attack_arguments_dict["3dcc_intensity"] != "nan" and not math.isnan(float(attack_arguments_dict["3dcc_intensity"])):
        attack_arguments_dict["3dcc_intensity"] = int(attack_arguments_dict["3dcc_intensity"])
    


    if "sintel" in row_dict["dataset"]:
        attack_arguments_dict["pretrained_ckpt"] = "sintel"
    else:
        attack_arguments_dict["pretrained_ckpt"] = "kitti"

    if row_dict["model"] == "ms_raft+":
        attack_arguments_dict["pretrained_ckpt"] = "mixed"
    
    # Build the combination string
    combination = f"{row_dict['model']} {attack_arguments_dict['pretrained_ckpt']} {attack_arguments_dict['val_dataset']} " \
                  f"{attack_arguments_dict['attack']} {attack_arguments_dict['attack_targeted']} {attack_arguments_dict['attack_target']} " \
                  f"{attack_arguments_dict['attack_loss']} {attack_arguments_dict['attack_epsilon']} {attack_arguments_dict['attack_alpha']} " \
                  f"{attack_arguments_dict['attack_iterations']} {attack_arguments_dict['attack_norm']} {attack_arguments_dict['cc_name']} " \
                  f"{attack_arguments_dict['cc_severity']} {attack_arguments_dict['attack_optim_target']} " \
                  f"{attack_arguments_dict['3dcc_corruption']} {attack_arguments_dict['3dcc_intensity']} "
    
    # Append the combination to the list
    combinations_list.append(f"\"{combination}\"")

# Determine the number of scripts required (80 combinations per script)
batch_size = 40
num_batches = math.ceil(len(combinations_list) / batch_size)

# Write the new scripts with combinations in batches of 80
for i in range(num_batches):
    # Create a new file name by adding a version number (v1, v2, v3, ...)
    script_vx_path_h100 = script_blueprint_path.replace("blueprint", f"v{i + 1}")

    # Start the combinations string for this batch
    combinations_content = "\ncombinations=("

    # Get the combinations for this batch
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(combinations_list))
    combinations_batch = combinations_list[start_idx:end_idx]

    # Append the combinations to the content
    combinations_content += " ".join(combinations_batch) + ")\n"

    # Write the new script with the header, combinations, and the rest of the script
    with open(script_vx_path_h100, 'w') as new_script:
        # Write the header first
        new_script.writelines(header_lines)
        
        # Write the combinations array
        new_script.write(combinations_content)
        
        # Write the rest of the original script
        new_script.writelines(rest_of_script)

    print(f"New SLURM script created at: {script_vx_path_h100}")



# Read the original blueprint script content
with open(script_blueprint_path_h100, 'r') as original_script:
    original_content = original_script.readlines()

# Split the header (lines starting with '#SBATCH') and the rest of the script
header_lines = []
rest_of_script = []
header_complete = False

for line in original_content:
    if line.startswith("#SBATCH") or line.startswith("#!/bin/bash") or not header_complete:
        header_lines.append(line)
        if line.strip() == "":
            header_complete = True
    else:
        rest_of_script.append(line)

# Initialize an empty list for combinations
combinations_list = []

# Iterate over the rows of df_not_in_a100_models to build the combinations
for index, row in df_in_a100_models.iterrows():
    attack_arguments_dict = {}
    row_dict = row.to_dict()
    
    # Build attack arguments
    attack_arguments_dict["val_dataset"] = row_dict.get("dataset", "nan")
    attack_arguments_dict["attack"] = row_dict.get("attack", "nan")
    attack_arguments_dict["attack_targeted"] = row_dict.get("targeted", "nan")
    attack_arguments_dict["attack_target"] = row_dict.get("target", "nan")
    attack_arguments_dict["attack_loss"] = row_dict.get("loss", "nan")
    attack_arguments_dict["attack_epsilon"] = row_dict.get("epsilon", "nan")

    # Adjust epsilon values based on conditions
    if attack_arguments_dict["attack_epsilon"] == 0.0314:
        attack_arguments_dict["attack_epsilon"] = round(8.0/255.0, 10)
    elif attack_arguments_dict["attack_epsilon"] == 0.0157:
        attack_arguments_dict["attack_epsilon"] = round(4.0/255.0, 10)
    elif attack_arguments_dict["attack_epsilon"] == 0.251:
        attack_arguments_dict["attack_epsilon"] = round(64.0/255.0, 10)

    attack_arguments_dict["attack_alpha"] = row_dict.get("alpha", "nan")
    attack_arguments_dict["attack_iterations"] = row_dict.get("iterations", "nan")
    attack_arguments_dict["attack_norm"] = row_dict.get("norm", "nan")
    attack_arguments_dict["cc_name"] = row_dict.get("name", "nan")
    attack_arguments_dict["cc_severity"] = row_dict.get("severity", "nan")
    attack_arguments_dict["attack_optim_target"] = row_dict.get("optim", "nan")
    attack_arguments_dict["3dcc_corruption"] = row_dict.get("3dcc_corruption", "nan")
    attack_arguments_dict["3dcc_intensity"] = row_dict.get("3dcc_intensity", "nan")

    if attack_arguments_dict["attack_iterations"] != "nan" and not math.isnan(float(attack_arguments_dict["attack_iterations"])):
        attack_arguments_dict["attack_iterations"] = int(attack_arguments_dict["attack_iterations"])
    if attack_arguments_dict["cc_severity"] != "nan" and not math.isnan(float(attack_arguments_dict["cc_severity"])):
        attack_arguments_dict["cc_severity"] = int(attack_arguments_dict["cc_severity"])
    if attack_arguments_dict["3dcc_intensity"] != "nan" and not math.isnan(float(attack_arguments_dict["3dcc_intensity"])):
        attack_arguments_dict["3dcc_intensity"] = int(attack_arguments_dict["3dcc_intensity"])

    if "sintel" in row_dict["dataset"]:
        attack_arguments_dict["pretrained_ckpt"] = "sintel"
    else:
        attack_arguments_dict["pretrained_ckpt"] = "kitti"

    if row_dict["model"] == "ms_raft+":
        attack_arguments_dict["pretrained_ckpt"] = "mixed"
    
    # Build the combination string
    combination = f"{row_dict['model']} {attack_arguments_dict['pretrained_ckpt']} {attack_arguments_dict['val_dataset']} " \
                  f"{attack_arguments_dict['attack']} {attack_arguments_dict['attack_targeted']} {attack_arguments_dict['attack_target']} " \
                  f"{attack_arguments_dict['attack_loss']} {attack_arguments_dict['attack_epsilon']} {attack_arguments_dict['attack_alpha']} " \
                  f"{attack_arguments_dict['attack_iterations']} {attack_arguments_dict['attack_norm']} {attack_arguments_dict['cc_name']} " \
                  f"{attack_arguments_dict['cc_severity']} {attack_arguments_dict['attack_optim_target']} " \
                  f"{attack_arguments_dict['3dcc_corruption']} {attack_arguments_dict['3dcc_intensity']} "
    
    # Append the combination to the list
    combinations_list.append(f"\"{combination}\"")

# Determine the number of scripts required (80 combinations per script)
batch_size = 40
num_batches = math.ceil(len(combinations_list) / batch_size)

# Write the new scripts with combinations in batches of 80
for i in range(num_batches):
    # Create a new file name by adding a version number (v1, v2, v3, ...)
    script_vx_path_h100 = script_blueprint_path_h100.replace("blueprint", f"v{i + 1}")

    # Start the combinations string for this batch
    combinations_content = "\ncombinations=("

    # Get the combinations for this batch
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(combinations_list))
    combinations_batch = combinations_list[start_idx:end_idx]

    # Append the combinations to the content
    combinations_content += " ".join(combinations_batch) + ")\n"

    # Write the new script with the header, combinations, and the rest of the script
    with open(script_vx_path_h100, 'w') as new_script:
        # Write the header first
        new_script.writelines(header_lines)
        
        # Write the combinations array
        new_script.write(combinations_content)
        
        # Write the rest of the original script
        new_script.writelines(rest_of_script)

    print(f"New SLURM script created at: {script_vx_path_h100}")


New SLURM script created at: /hkfs/work/workspace/scratch/ma_jcaspary-ma_jcaspary-team_project_fss2024/benchmarking_robustness/optical_flow_estimation/scripts/evaluation_minimum/automated_script/automated_script/automatic_script_v1.sh
New SLURM script created at: /hkfs/work/workspace/scratch/ma_jcaspary-ma_jcaspary-team_project_fss2024/benchmarking_robustness/optical_flow_estimation/scripts/evaluation_minimum/automated_script/automated_script/automatic_script_v2.sh
New SLURM script created at: /hkfs/work/workspace/scratch/ma_jcaspary-ma_jcaspary-team_project_fss2024/benchmarking_robustness/optical_flow_estimation/scripts/evaluation_minimum/automated_script/automated_script/automatic_script_v3.sh
New SLURM script created at: /hkfs/work/workspace/scratch/ma_jcaspary-ma_jcaspary-team_project_fss2024/benchmarking_robustness/optical_flow_estimation/scripts/evaluation_minimum/automated_script/automated_script/automatic_script_v4.sh
New SLURM script created at: /hkfs/work/workspace/scratch/ma